In [1]:
# coding='utf-8'
import os
import sys
import numpy as np
import time
import datetime
import json
import importlib
import logging
import shutil
import random

# import matplotlib
# matplotlib.use('Agg')
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# from matplotlib.ticker import NullLocator
# from PIL import Image
import torch
import torch.nn as nn

from yolo_model import yoloModel
from PASCAL_Dataloader import create_split_loaders
from YOLO_Loss import YoloLoss
from utils import NMS, remove_nulls, convert_to_xywh
from bbox import non_max_suppression
from detect import plot_detections

%matplotlib inline

In [2]:
config = {}
config["batch_size"] = 16
config['backbone_name'] = "darknet_53"
config['backbone_pretrained'] = ""
config['anchors'] = [[[116, 90], [156, 198], [373, 326]],
                            [[30, 61], [62, 45], [59, 119]],
                            [[10, 13], [16, 30], [33, 23]]]
config['classes'] = 20
config['img_h'] = config['img_w'] = 416
config['confidence_threshold'] = 0.5
config['nms_threshold'] = 0.4

'''
UPDATE THESE VALUES!

'''
config['pretrain_snapshot'] = "./states/20190531004422/model.pth" # "./states/20190530222509/model.pth"
config['root_dir'] = './VOC2012'
config['output_dir'] = './detected_images/'

In [3]:
CLASS_ID = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", \
            "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", \
            "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

In [5]:
_, _, test_loader = create_split_loaders(root_dir=config['root_dir'], batch_size=config['batch_size'])
print('There are %d batches in test set. This only runs the first 15 batches.' %len(test_loader))
print('To run the entire test set, set is_break to False')

There are 130 batches in test set. This only runs the first 15 batches.
To run the entire test set, set is_break to False


In [6]:
def test(config, test_loader, output_dir=config['output_dir'], is_break=True):
    # Creates the output folder for the results if not already created.
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
    
    # Load and initialize network
    net = yoloModel(config).cuda()

    # Restore pretrain model
    if config["pretrain_snapshot"]:
        print('Loading model...')
        logging.info("load checkpoint from {}".format(config["pretrain_snapshot"]))
        state_dict = torch.load(config["pretrain_snapshot"])
        net.load_state_dict(state_dict)
    else:
        raise Exception("missing pretrain_snapshot!!!")


    # YOLO loss with 3 scales
    yolo_losses = []
    for i in range(3):
        yolo_losses.append(YoloLoss(config["classes"], (config["img_w"], config["img_h"]), config["anchors"][i]))
        
    # Start inference
    print('Ready for detections!')             
    for batch, sample in enumerate(test_loader):
        if batch >=15 and is_break:
            print('\nbreaking')
            break
            
        images, labels = sample["image"], sample["label"]
        orig_W, orig_H = sample["orig_img"][0], sample["orig_img"][1]
        target_labels = sample["target_label"]
        imgfiles = sample["imgfile"]
        
        # Create image pathfiles
        imgpaths = [os.path.join(config["root_dir"], 'JPEGImages', x) for x in imgfiles]
        
        orig_W = orig_W.to('cpu').numpy()
        orig_H = orig_H.to('cpu').numpy()
        images = images.cuda()
        
        # Run images through model, get predictions, and apply NMS.
        with torch.no_grad():
            outputs = net(images)
            output_list = []
            for i in range(3):
                output_list.append(yolo_losses[i](outputs[i]))
            detections = torch.cat(output_list, 1)
            detections = non_max_suppression(detections, 20)

        for ii, detection in enumerate(detections): ## 0 to 15
        # ------------------------------------- #
        # Parsing targets and detections labels #
        # ------------------------------------- #
            print('\nbatch %d, sample %d' %(batch,ii))
            targets = convert_to_xywh(remove_nulls(target_labels[ii,:].to('cpu').numpy())) #70x5
            print('target labels (cls, x, y, w, h)\n', targets)           
            
            if detection is not None:
                # Rescale coordinates to original dimensions
                RATIO_W = orig_W[ii] / config["img_w"]
                RATIO_H = orig_H[ii] / config["img_h"]
            
                predictions = np.zeros((len(detection),5), dtype=int)
                for jj, (x1, y1, x2, y2, conf, cls_conf, cls_pred) in enumerate(detection):        
                    x = int(x1 * RATIO_W)
                    y = int(y1 * RATIO_H)
                    w = int(abs(x2 - x1) * RATIO_W)
                    h = int(abs(y2 - y1) * RATIO_H)
                    predictions[jj,:] = (int(cls_pred), x, y, w, h) 
                print('predicted labels (cls, x, y, w, h)\n', predictions)
                
            else:
                predictions = []
                print('predicted labels (cls, x, y, w, h) - No objects detected!!')
            
             # ----------------------------------------- #
            # Plot targets and detections bounding boxes #
            # ------------------------------------------ #
            plot_detections(imgpaths[ii], targets, predictions, \
                        batch=batch, sample=ii, output_dir=output_dir, is_demo=False)
    
    print('Finished object detections! Check the results.')
    
#         # ------------------- #
#         # Plot bounding boxes #
#         # ------------------- #
#             plt.figure()
# #             fig, ax = plt.subplots(1)
#             fig, ax = plt.subplots(1, 2, figsize=(8,4))
#             img = Image.open(imgpaths[ii]).convert('RGB')
#             ax[0].imshow(img)
#             ax[1].imshow(img)
#             ax[0].set_title('Groundtruth Bounding Boxes')
#             ax[1].set_title('Predicted Bounding Boxes')
#             ax[0].axis('off')
#             ax[1].axis('off')
            
#             # -------------------------- #
#             # Plotting groundtruth boxes #
#             # -------------------------- #
#             for jj in range(len(targets)):
#                 cls, x, y, w, h = targets[jj,0], targets[jj,1], targets[jj,2], targets[jj,3], targets[jj,4]
#                 color = colors[cls]
#                 # Create a Rectangle patches
#                 bbox = patches.Rectangle((x, y), w, h, linewidth=2, \
#                                          edgecolor=color, facecolor='none')
#                 # Add the bbox to the plot
#                 ax[0].add_patch(bbox)
#                 # Add label
#                 ax[0].text(x, y, s=CLASS_ID[cls], color='white', \
#                          verticalalignment='top',bbox={'color': color, 'pad': 0})
            
            
#             # ------------------------ #
#             # Plotting predicted boxes #
#             # ------------------------ #
#             for jj in range(len(predictions)):
#                 cls, x, y, w, h = predictions[jj,0], predictions[jj,1], \
#                                   predictions[jj,2], predictions[jj,3], predictions[jj,4]
#                 color = colors[cls+20]
#                 # Create a Rectangle patches
#                 bbox = patches.Rectangle((x, y), w, h, linewidth=2, \
#                                          edgecolor=color, facecolor='none')
#                 # Add the bbox to the plot
#                 ax[1].add_patch(bbox)
#                 # Add label
#                 ax[1].text(x, y, s=CLASS_ID[cls], color='black', \
#                          verticalalignment='top',bbox={'color': color, 'pad': 0})
                
#             # Save generated image with detections
#             plt.tight_layout()
#             plt.gca().xaxis.set_major_locator(NullLocator())
#             plt.gca().yaxis.set_major_locator(NullLocator())
#             outputfile = output_dir + '/{}_{}.jpg'.format(batch, ii)
#             plt.savefig(outputfile, bbox_inches='tight', pad_inches=0.0)
#             plt.close()
            


In [7]:
if __name__ == "__main__":
    test(config,test_loader)

Loading model...
Ready for detections!

batch 0, sample 0
target labels (cls, x, y, w, h)
 [[  9  17  86 265 359]]
predicted labels (cls, x, y, w, h)
 [[  9  24  84 260 392]]

batch 0, sample 1
target labels (cls, x, y, w, h)
 [[  8 213   1  97 147]
 [  8 295  22 100 144]
 [ 10 223   1 178 129]]
predicted labels (cls, x, y, w, h)
 [[  8 299  35 103 120]]

batch 0, sample 2
target labels (cls, x, y, w, h)
 [[  2  65  71 407 235]]
predicted labels (cls, x, y, w, h) - No objects detected!!

batch 0, sample 3
target labels (cls, x, y, w, h)
 [[ 15   1   1 499 374]]
predicted labels (cls, x, y, w, h) - No objects detected!!

batch 0, sample 4
target labels (cls, x, y, w, h)
 [[ 14  89 124 111 211]
 [ 14 323 145  45 119]]
predicted labels (cls, x, y, w, h)
 [[ 14 343 182  34  72]]

batch 0, sample 5
target labels (cls, x, y, w, h)
 [[  7   1  78 374 422]
 [ 19   1  24 106 160]]
predicted labels (cls, x, y, w, h) - No objects detected!!

batch 0, sample 6
target labels (cls, x, y, w, h)
 [[ 1

/opt/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)



batch 1, sample 4
target labels (cls, x, y, w, h)
 [[  1  89 175  92 123]
 [  1 192 171 101 111]
 [ 14 197  89  96 150]
 [ 14  86  95  85 170]
 [ 14  98  77  49  55]]
predicted labels (cls, x, y, w, h)
 [[  1 100 158  69 115]
 [ 14 106 117  44 146]
 [ 14 214  93  61 151]]

batch 1, sample 5
target labels (cls, x, y, w, h)
 [[ 13  70  98 201 365]]
predicted labels (cls, x, y, w, h) - No objects detected!!

batch 1, sample 6
target labels (cls, x, y, w, h)
 [[  9 129   1 288 285]]
predicted labels (cls, x, y, w, h) - No objects detected!!

batch 1, sample 7
target labels (cls, x, y, w, h)
 [[ 14 151  45 271 289]
 [ 14 301  77 199 260]
 [  3  36 261 464 114]]
predicted labels (cls, x, y, w, h)
 [[ 14 156  14 179 347]]

batch 1, sample 8
target labels (cls, x, y, w, h)
 [[ 11  35   1 465 374]]
predicted labels (cls, x, y, w, h) - No objects detected!!

batch 1, sample 9
target labels (cls, x, y, w, h)
 [[ 11   1   4 140 109]
 [  7 387  68 106  52]]
predicted labels (cls, x, y, w, h)
 [[  


batch 4, sample 2
target labels (cls, x, y, w, h)
 [[ 14 433 159  44 110]
 [  6  65 158 380 125]
 [ 14 284 148  21  45]
 [  6   1 159 192  76]
 [  6   1 158  74  29]]
predicted labels (cls, x, y, w, h)
 [[  6  -2 137 192 102]
 [  6  83 132 318 161]]

batch 4, sample 3
target labels (cls, x, y, w, h)
 [[ 14   1   5 272 370]
 [ 14 253  51 247 313]]
predicted labels (cls, x, y, w, h) - No objects detected!!

batch 4, sample 4
target labels (cls, x, y, w, h)
 [[  0 164  14 167 127]]
predicted labels (cls, x, y, w, h) - No objects detected!!

batch 4, sample 5
target labels (cls, x, y, w, h)
 [[  5 222 187 189 117]]
predicted labels (cls, x, y, w, h)
 [[  5 223 175 201 123]]

batch 4, sample 6
target labels (cls, x, y, w, h)
 [[  1  93  67 332 227]]
predicted labels (cls, x, y, w, h)
 [[  1 113 130 275 177]]

batch 4, sample 7
target labels (cls, x, y, w, h)
 [[  7   1   1 499 332]]
predicted labels (cls, x, y, w, h)
 [[  7   9  35 482 256]]

batch 4, sample 8
target labels (cls, x, y, w, 


batch 6, sample 14
target labels (cls, x, y, w, h)
 [[ 11   1  26 345 474]]
predicted labels (cls, x, y, w, h) - No objects detected!!

batch 6, sample 15
target labels (cls, x, y, w, h)
 [[  9  23 183 210  91]
 [  9  32  93 323 169]]
predicted labels (cls, x, y, w, h) - No objects detected!!

batch 7, sample 0
target labels (cls, x, y, w, h)
 [[  0   1  35 486 258]
 [ 14 385 236  21  62]
 [ 14 329 245  17  49]
 [ 14 353 239  23  59]
 [ 14 356 239  24  51]
 [ 14 480 239   7  18]
 [ 14 471 236   7  21]
 [ 14 466 237   7  18]
 [ 14 455 236   9  19]
 [ 14 461 238   5  18]
 [ 14 439 237   6  16]
 [ 14 435 237   6  16]
 [ 14 426 239   6  14]
 [ 14 412 238   6  17]
 [ 14 416 237   6  15]
 [ 14 420 239   3  12]
 [ 14 422 237   4  15]
 [ 14 382 235   7  15]
 [ 14 375 235   6  13]]
predicted labels (cls, x, y, w, h)
 [[  0  24  67 447 184]
 [ 14 462 226  13  29]
 [ 14 401 239  15  21]
 [ 14 437 236  12  25]
 [ 14 428 241  23  56]
 [ 14 384 244  16  40]
 [ 14 444 237  15  24]
 [ 14 359 239  23 

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>